1. Clone diffusion model's repo

In [3]:
!git clone https://github.com/huggingface/diffusers.git

Cloning into 'diffusers'...
remote: Enumerating objects: 111497, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 111497 (delta 59), reused 33 (delta 28), pack-reused 111403 (from 2)
Receiving objects: 100% (111497/111497), 83.96 MiB | 28.61 MiB/s, done.
Resolving deltas: 100% (83182/83182), done.


2. Install required packages

In [4]:
%cd /content/diffusers/
!ls -al
!pip install -e .

/content/diffusers
total 176
drwxr-xr-x 12 root root  4096 Nov 20 18:58 .
drwxr-xr-x  1 root root  4096 Nov 20 18:58 ..
drwxr-xr-x  2 root root  4096 Nov 20 18:58 benchmarks
-rw-r--r--  1 root root  1325 Nov 20 18:58 CITATION.cff
-rw-r--r--  1 root root  5324 Nov 20 18:58 CODE_OF_CONDUCT.md
-rw-r--r--  1 root root 39267 Nov 20 18:58 CONTRIBUTING.md
drwxr-xr-x  9 root root  4096 Nov 20 18:58 docker
drwxr-xr-x  3 root root  4096 Nov 20 18:58 docs
drwxr-xr-x 25 root root  4096 Nov 20 18:58 examples
drwxr-xr-x  8 root root  4096 Nov 20 18:58 .git
drwxr-xr-x  5 root root  4096 Nov 20 18:58 .github
-rw-r--r--  1 root root  1910 Nov 20 18:58 .gitignore
-rw-r--r--  1 root root 11357 Nov 20 18:58 LICENSE
-rw-r--r--  1 root root  2782 Nov 20 18:58 Makefile
-rw-r--r--  1 root root    67 Nov 20 18:58 MANIFEST.in
-rw-r--r--  1 root root 15348 Nov 20 18:58 PHILOSOPHY.md
-rw-r--r--  1 root root   785 Nov 20 18:58 pyproject.toml
-rw-r--r--  1 root root 14402 Nov 20 18:58 README.md
drwxr-xr-x  2 root r

In [5]:
%cd /content/diffusers/examples/text_to_image
!pip install -r requirements_sdxl.txt

/content/diffusers/examples/text_to_image
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.8 MB/s eta 0:00:00


3. hugging face login

In [6]:
# !huggingface-cli login
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `413 genshin!` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenti

4. for weight and bias

In [7]:
!pip install wandb
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mary4net (mary4net-university-of-toronto) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [8]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
Tesla T4


In [10]:
from google.colab import drive
drive.mount('/content/drive')

!ls /content/drive/MyDrive/UTM/model


Mounted at /content/drive


In [13]:
!accelerate launch train_text_to_image_lora_sdxl.py \
 --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --pretrained_vae_model_name_or_path="madebyollin/sdxl-vae-fp16-fix" \
  --dataset_name="mary-ruiliii/Genshin-character-portrait-image" \
  --image_column="image" \
  --caption_column="text" \
  --validation_prompt="" \
  --num_validation_images=0 \
  --validation_epochs=999999 \
  --resolution=318 \
  --random_flip \
  --train_batch_size=1 \
  --num_train_epochs=10 \
  --checkpointing_steps=500 \
  --gradient_accumulation_steps=4 \
  --learning_rate=1e-04 \
  --lr_warmup_steps=0 \
  --report_to="wandb" \
  --resume_from_checkpoint="latest" \
  --dataloader_num_workers=8 \
  --allow_tf32 \
  --mixed_precision="fp16" \
  --push_to_hub \
  --output_dir=/content/drive/MyDrive/UTM/model \
  --hub_model_id="genshin-style_character_generator"

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-11-20 20:27:24.085838: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763670444.114850   27119 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763670444.123609   27119 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763670444.158624   27119 computation_plac

In [2]:
from diffusers import DiffusionPipeline
import torch

# Load Stable Diffusion XL Base1.0
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to("cuda")

# Optional CPU offloading to save some GPU Memory
pipe.enable_model_cpu_offload()

# Loading Trained DreamBooth LoRA Weights
pipe.load_lora_weights("mary-ruiliii/genshin-style_character_generator")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
prompt = "栗黎尔"

# Invoke pipeline to generate image
image = pipe(
    prompt = prompt,
    num_inference_steps=50,
    height=1024,
    width=1024,
    guidance_scale=7.0,
).images[0]